# Notebook 5: Learning Planner Agent - Development and Testing

### Objective
The purpose of this notebook is to develop and test our third agent: the `Learning Planner`. This agent uses an LLM to find and curate free, high-quality learning resources for a given list of skill gaps.

### Key Steps:
1.  **Define Inputs**: We will start with a sample list of skill gaps, representing the output from our `Gap Analyst`.
2.  **Define Output Structure**: We'll create Pydantic models to ensure the agent's output is structured and reliable.
3.  **Develop the Prompt**: Craft a detailed prompt to instruct the LLM to act as a learning mentor.
4.  **Implement and Test Agent Logic**: Implement the agent function in its `.py` file and test it here by calling it with our sample inputs.

In [1]:
import sys
import os
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field, HttpUrl
from typing import List, Dict
from tqdm.auto import tqdm


# Add project root to path to allow importing from src
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


from src.agents import learning_planner

# This is a sample list of skill gaps, simulating the output from Phase 2.
skill_gaps_to_learn = [
    'kubernetes',
    'go',
    'terraform',
    'google cloud platform (gcp)',
    'ci/cd'
]

c:\Users\gusta\OneDrive\Documentos\GitHub\agentic-skill-mapper\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Developing the Agent's Core Logic

The intelligence of our `Learning Planner` resides almost entirely in its prompt. Our goal is to create a set of instructions that guides the LLM to perform several tasks:
1.  Act as an expert learning guide.
2.  Find a variety of **free, high-quality** learning resources.
3.  Categorize each resource (e.g., Video, Article, Course).
4.  Return the findings in a structured JSON format that matches our Pydantic models.

We will define this logic within a function in this notebook for rapid, iterative testing.

In [2]:
class LearningResource(BaseModel):
    title: str = Field(description="The descriptive title of the resource.")
    url: HttpUrl = Field(description="The direct URL to the resource.")
    resource_type: str = Field(description="The type of the resource, e.g., 'Video', 'Article', 'Tutorial', 'Official Docs'.")

class LearningPlan(BaseModel):
    """A structured learning plan containing a list of resources for a skill."""
    plan: List[LearningResource] = Field(description="A list of curated learning resources.")


def find_resources_for_skill(skill_name: str) -> LearningPlan:
    """Finds a curated list of free learning resources for a single skill using an LLM."""
    
    parser = JsonOutputParser(pydantic_object=LearningPlan)
    
    prompt_template = """
    You are an expert curriculum developer and learning mentor for software engineers.
    Your task is to find the best **free** and **stable** online resources to learn a specific technical skill.

    For the skill "{skill_name}", find 3 to 4 high-quality, free-to-access learning resources.

    RULES:
    1.  **Prioritize Stable URLs**: Your knowledge has a cutoff date. To avoid 404 errors, prioritize links to high-level pages like the main documentation site (e.g., 'react.dev/learn'), a main tutorials page (e.g., 'kubernetes.io/docs/tutorials'), or a search query on a reputable platform (e.g., 'youtube.com/results?search_query=learn+terraform'). Avoid deep links to specific, obscure blog posts or old tutorials.
    2.  **Free Only**: All resources must be 100% free.
    3.  **Variety**: Provide a mix of resource types (e.g., 'Official Docs', 'Video Search', 'Tutorials Hub').
    4.  **High Quality**: Prioritize official documentation and well-known educational platforms.
    5.  **JSON Output**: You MUST return your response *only* as a valid JSON object with a single key "plan".

    {format_instructions}
    """

    llm = OllamaLLM(model="llama3:8b", base_url="http://127.0.0.1:11434", temperature=0.1)
    
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["skill_name"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    chain = prompt | llm | parser

    try:
        print(f"Agent: Searching for STABLE resources for '{skill_name}'...")
        result = chain.invoke({"skill_name": skill_name})
        return LearningPlan(**result)
    except Exception as e:
        print(f"--- AGENT ERROR ---")
        print(f"An exception occurred while searching for '{skill_name}'. Details: {e}")
        return LearningPlan(plan=[])



### Testing the Agent on a Single Skill

Before building the full loop, let's test our function on a single skill from our `skill_gaps_to_learn` list. This allows us to quickly validate our prompt and the LLM's output. We'll start with 'kubernetes'.

In [3]:
target_skill = skill_gaps_to_learn[0]

learning_plan = find_resources_for_skill(target_skill)

if learning_plan and learning_plan.plan:
    print(f"\nFound {len(learning_plan.plan)} resources for '{target_skill.title()}':")
    for res in learning_plan.plan:
        print(f"  - Title: {res.title}")
        print(f"    Type: {res.resource_type}")
        print(f"    URL: {res.url}")
else:
    print(f"\nNo resources were found for '{target_skill.title()}'.")

Agent: Searching for STABLE resources for 'kubernetes'...

Found 3 resources for 'Kubernetes':
  - Title: Official Kubernetes Documentation
    Type: Official Docs
    URL: https://kubernetes.io/docs/
  - Title: Kubernetes Tutorials Hub
    Type: Tutorials Hub
    URL: https://kubernetes.io/docs/tutorials/
  - Title: Kubernetes YouTube Search Results
    Type: Video Search
    URL: https://www.youtube.com/results?search_query=learn+kubernetes


## Creating the Full Learning Plan

Now that our core function `find_resources_for_skill` is robust, we need a main function to orchestrate the process for all the skills in our `skill_gaps_to_learn` list. This new function will loop through each skill, call our agent, and compile the results into a single, comprehensive `LearningPlan` object.

In [4]:
def create_full_learning_plan(skills_to_learn: List[str]) -> Dict[str, List[LearningResource]]:
    """
    Generates a comprehensive learning plan for a list of skill gaps.
    """
    full_plan_dict = {}
    
    for skill in tqdm(skills_to_learn, desc="Generating Learning Plan"):
        resources_plan = find_resources_for_skill(skill)
        
        if resources_plan and resources_plan.plan:
            full_plan_dict[skill] = resources_plan.plan
        else:
            full_plan_dict[skill] = []
            
    return full_plan_dict



### Executing the Full Pipeline

In [5]:
final_learning_plan_dict = create_full_learning_plan(skill_gaps_to_learn)

print("\n" + "="*50)
print("     C U S T O M I Z E D   L E A R N I N G   P L A N")
print("="*50 + "\n")

if final_learning_plan_dict:
    for skill, resources in final_learning_plan_dict.items():
        print(f" Skill to Learn: {skill.title()}")
        if resources:
            for res in resources:
                print(f"  - Title: {res.title}")
                print(f"    Type: {res.resource_type}")
                print(f"    URL: {res.url}")
        else:
            print("  - No resources were found for this skill (or an error occurred).")
        print("-" * 20)
else:
    print("The learning plan is empty.")

Generating Learning Plan:   0%|          | 0/5 [00:00<?, ?it/s]

Agent: Searching for STABLE resources for 'kubernetes'...


Generating Learning Plan:  20%|██        | 1/5 [01:27<05:51, 87.95s/it]

Agent: Searching for STABLE resources for 'go'...


Generating Learning Plan:  40%|████      | 2/5 [04:01<06:20, 126.70s/it]

...

Agent: Searching for STABLE resources for 'google cloud platform (gcp)'...


Generating Learning Plan:  80%|████████  | 4/5 [09:44<02:37, 157.85s/it]

Agent: Searching for STABLE resources for 'ci/cd'...


Generating Learning Plan: 100%|██████████| 5/5 [12:24<00:00, 148.80s/it]

--- AGENT ERROR ---
An exception occurred while searching for 'ci/cd'. Details: 1 validation error for LearningPlan
plan.1.resource_type
  Field required [type=missing, input_value={'title': 'CircleCI Offic...eci.com/docs/,\n      '}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

     C U S T O M I Z E D   L E A R N I N G   P L A N

 Skill to Learn: Kubernetes
  - Title: Official Kubernetes Documentation
    Type: Official Docs
    URL: https://kubernetes.io/docs/
  - Title: Kubernetes Tutorials Hub
    Type: Tutorials Hub
    URL: https://kubernetes.io/docs/tutorials/
  - Title: Kubernetes YouTube Channel
    Type: Video Search
    URL: https://www.youtube.com/results?search_query=learn+kubernetes
--------------------
 Skill to Learn: Go
  - Title: Official Go Documentation
    Type: Official Docs
    URL: https://golang.org/doc/
  - Title: Go Tutorials by Google
    Type: Tutorials Hub
    URL: https://tour.golang.org/list
  - Title: G

## Final Step: Modularize the Agent

The development and validation of our `Learning Planner` agent are now complete. The final step is to move the complete logic (Pydantic models and both functions) into its official project file, `src/agents/learning_planner.py`.